In [1]:
# Part 1: Import Libraries and Fetch Data
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

# Set style for plots
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Select stock 
stock_symbol = 'TSLA'  # Tesla
# stock_symbol = 'AAPL'  # Apple
# stock_symbol = 'MSFT'  # Microsoft
# stock_symbol = 'GOOGL' # Google

print(f"Downloading data for {stock_symbol}...")

# Download historical data
stock_data = yf.download(stock_symbol, period='5y')
print(f"Downloaded {len(stock_data)} days of data")
print("\nFirst few rows of data:")
print(stock_data.head())

ModuleNotFoundError: No module named 'yfinance'